In [ ]:
import torch
import warnings
from math import comb
import sys
import os
warnings.filterwarnings('ignore')

project_root = os.path.abspath("..")  
if project_root not in sys.path:
    sys.path.insert(0, project_root)
from src.rl_environment import RLEnvironment, defining_environments
from src.rl_policy import UnifiedPolicy, Policy
from src.cortical_estimator import CORTICAL
from src.ba_estimator import MI_ESTIMATOR
from src.simulation_runner import runing_sims

In [ ]:
dimension = 2
num_thresholds = 4
alphabet_size = int(sum(comb(num_thresholds, k) for k in range(dimension + 1)))
box_param = 1.5

max_steps, patience, norm_patience = 1000, 50, (50,1000)
ln_steps, pt_steps = 50, 50
mi_est_name = 'BA'

channel_type = 'changing-csi-smooth-0.05'
if mi_est_name == 'BA': mi_est = MI_ESTIMATOR(dimension, box_param, channel_type,10000)
else:
    mi_est = CORTICAL((512,dimension,alphabet_size,num_thresholds,10),alphas=[1.0, 0.5, 0.1, 0.01],
                  lambda_entropy=0.3,cost_coef=10.0,box_param=box_param)
    mi_est.load_models('./models/cortical_models/')

thrsh = None
if dimension==1.0 and num_thresholds==1: thrsh = torch.tensor([1.0,0.0])
elif dimension==1.0 and num_thresholds==2: thrsh = torch.tensor([1.0,-0.5,1.0,0.5])
elif dimension==1.0 and num_thresholds==3: thrsh = torch.tensor([1.0,-0.5,1.0,0.0,1.0,0.5])
elif dimension==2.0 and num_thresholds==2: thrsh = torch.tensor([1.0,0.0,0.0,0.0,1.0,0.0])
elif dimension==2.0 and num_thresholds==3: thrsh = torch.tensor([0.0,1.0,0.0,1.0,0.0,-0.25,1.0,0.0,0.25])
elif dimension==2.0 and num_thresholds==4: thrsh = torch.tensor([1.0,0.0,-0.25,1.0,0.0,0.25,0.0,1.0,0.25,0.0,1.0,-0.25])

run_id = 'run10'
policy = torch.load(f"./models/policy_models/unified_policy_{mi_est_name}-{dimension}D-{num_thresholds}-{run_id}.pth")
for sim_count in range(10):
    runing_sims(dimension,num_thresholds,alphabet_size,box_param,max_steps,patience,norm_patience,ln_steps,pt_steps,policy,mi_est,run_id,sim_count,thrsh)